# Rules

In [1]:
import proportional_ranking as pr
import numpy as np
import pandas as pd
np.random.seed(42)

## Generate a profile

Let's first see how to generate a profile of voter. A profile is represented as a numpy matrix of boolean such that $P[i,j]$ is *True* if and only if voter $i$ approve candidate $j$.

In [2]:
n_voters = 10
n_candidates = 6 
profile = pr.generate_profile(n_voters, n_candidates)
profile

array([[False,  True,  True,  True, False, False],
       [False,  True,  True,  True, False,  True],
       [ True, False, False, False, False,  True],
       [False, False,  True, False, False, False],
       [False,  True, False,  True,  True, False],
       [ True, False, False,  True,  True,  True],
       [False, False,  True, False, False, False],
       [False,  True, False,  True, False,  True],
       [ True, False,  True,  True,  True,  True],
       [ True,  True, False, False, False, False]])

We can also print the profile with *print_profile*

In [3]:
pr.print_profile(profile)

0 : b c d
1 : b c d f
2 : a f
3 : c
4 : b d e
5 : a d e f
6 : c
7 : b d f
8 : a c d e f
9 : a b


# Find a proportional ranking

You can use any of the rule to create a proportional ranking, print the ranking, and compute the quality of the ranking. For instance, with approval voting :

In [4]:
approval_voting = pr.AV()
approval_voting.set_profile(profile)

In [5]:
approval_voting.ranking()

array([3, 5, 2, 1, 0, 4], dtype=int64)

In [6]:
approval_voting.print_ranking()

d > f > c > b > a > e


In [7]:
approval_voting.quality

1.0

# The different voting rules

Let's compare some voting rule. First, rules from the original paper :

In [8]:
rules = [pr.AV(), 
         pr.reversePAV(), 
         pr.phragmenMinmax(),
         pr.seqPAV(), 
         pr.geometric(2), 
         pr.geometric(5/4), 
         pr.geometric(10)]

In [9]:
results = pr.compare_rules(8, 5, rules, 1000)

c:\users\theo delemazure\documents\ens\stage berlin\github\proportional_ranking\proportional_ranking\rules\reversePAV.py:29: RuntimeWarning: divide by zero encountered in true_divide
  s = (1 / curr_weights).dot(scores)
c:\users\theo delemazure\documents\ens\stage berlin\github\proportional_ranking\proportional_ranking\rules\Phragmen.py:20: RuntimeWarning: divide by zero encountered in true_divide
  s = (1 + load.dot(scores)) / (np.ones(n).dot(scores))
c:\users\theo delemazure\documents\ens\stage berlin\github\proportional_ranking\proportional_ranking\rules\Phragmen.py:24: RuntimeWarning: invalid value encountered in double_scalars
  if min_v - s[k] > 0.0001:


In [10]:
avg, fails = results
ranking = np.argsort(avg)
rules_name = [rules[i].name for i in ranking[:-1:]]
rules_name.append("BestOf")
avg_sorted = avg[ranking]
fails_sorted = fails[ranking]
M = np.zeros((2,len(ranking)))
M[0] = avg_sorted
M[1] = fails_sorted
pd.DataFrame(M.T, columns=["quality", "success"], index=rules_name)
    

,quality,success
AV,1.074864,0.947
geometric (10.00),1.084042,0.978
geometric (2.00),1.086878,0.979
reversePAV,1.089883,0.978
geometric (1.25),1.093725,0.983
seqPAV,1.095592,0.985
PhragmenMinmax,1.098456,0.983
BestOf,1.108242,0.994


Let's see now how my personnal rules perform : 

In [11]:
rules = [pr.AV(), 
         pr.phragmenMinmax(),
         pr.seqPAV(),
         pr.geometric(5/4),
        pr.reversePAV(.01),
        pr.reversePAV(-.001),
        pr.seqPAV(.01),
        pr.seqPAV(-.1)]
results = pr.compare_rules(8, 5, rules, 1000)

avg, fails = results
ranking = np.argsort(avg)
rules_name = [rules[i].name for i in ranking[:-1:]]
rules_name.append("BestOf")
avg_sorted = avg[ranking]
fails_sorted = fails[ranking]
M = np.zeros((2,len(ranking)))
M[0] = avg_sorted
M[1] = fails_sorted
pd.DataFrame(M.T, columns=["quality", "success"], index=rules_name)

,quality,success
AV,1.099306,0.959
geometric (1.25),1.116950,0.986
seqPAV (alpha = -0.10),1.117200,0.986
reversePAV (alpha = -0.00),1.117742,0.991
seqPAV,1.118658,0.987
seqPAV (alpha = 0.01),1.119258,0.984
PhragmenMinmax,1.119503,0.987
reversePAV (alpha = 0.01),1.120033,0.989
BestOf,1.130681,0.996


You can see that the success rate seems to be a bit higher with a $\alpha \ne 0$ for reversePAV. Now let's see how bordaPAV performs, and the maximal success rate (which is acheived by *justifyIt*, a rule that simply search for a ranking with quality $> 1$).

In [12]:
rules = [pr.AV(), 
         pr.phragmenMinmax(),
         pr.seqPAV(),
        pr.bordaPAV(),
        pr.reversePAV(.01),
        pr.reversePAV(-.001),
        pr.justifyIt()]
results = pr.compare_rules(8, 5, rules, 1000)

avg, fails = results
ranking = np.argsort(avg)
rules_name = [rules[i].name for i in ranking[:-1:]]
rules_name.append("BestOf")
avg_sorted = avg[ranking]
fails_sorted = fails[ranking]
M = np.zeros((2,len(ranking)))
M[0] = avg_sorted
M[1] = fails_sorted
pd.DataFrame(M.T, columns=["quality", "success"], index=rules_name)

,quality,success
JustifyIt,1.029831,0.997
AV,1.085928,0.961
reversePAV (alpha = -0.00),1.101392,0.989
seqPAV,1.103967,0.987
bordaPAV,1.104842,0.990
reversePAV (alpha = 0.01),1.105903,0.989
PhragmenMinmax,1.106214,0.988
BestOf,1.115311,0.997


# A Hard case

There exists some "hard" profile, like this one :

In [2]:
profile = pr.hard_profile_1
pr.print_profile(profile)

0 : a d
1 : a d
2 : a b
3 : a b c e
4 : b d
5 : b c d e
6 : c d
7 : c e


None of the rules return a ranking with quality $> 1$ for this profile, even if such a ranking exists, as it is shown by the rule *maxQuality*. It can be due to tie-breaking mechanism, but that shows that all these rules are not **proportional-consistant**.

In [5]:
pr.test_profile(profile, [pr.AV(), 
         pr.phragmenMinmax(),
         pr.seqPAV(),
         pr.geometric(5/4),
        pr.reversePAV(.01),
        pr.reversePAV(-.001),
        pr.seqPAV(.01),
        pr.seqPAV(-.1),
         pr.reversePAV(), 
         pr.geometric(2), 
         pr.geometric(10),
        pr.phragmenClassic(),
        pr.IRVSum(),
        pr.phragmenDepile(),
        pr.bordaPAV(),
        pr.maxQuality()])

AV : 0.75
d > c > b > a > e
PhragmenMinmax : 0.75
d > a > c > b > e
seqPAV : 0.75
d > a > c > b > e
geometric (1.25) : 0.75
d > a > c > b > e
reversePAV (alpha = 0.01) : 0.75
d > a > c > b > e
reversePAV (alpha = -0.00) : 0.75
d > a > c > b > e
seqPAV (alpha = 0.01) : 0.75
d > a > c > b > e
seqPAV (alpha = -0.10) : 0.75
d > a > c > b > e
reversePAV : 0.75
a > d > c > b > e
geometric (2.00) : 0.75
d > a > c > b > e
geometric (10.00) : 0.75
d > a > c > b > e
PhragmenClassic : 0.75
d > a > c > b > e
IRVSum : 0.75
d > a > c > b > e
PhragmenDepile : 0.75
d > a > c > b > e
bordaPAV : 0.75
d > a > c > b > e
MaxQuality : 1.00
b > d > a > c > e
